In [ ]:
import time

In [ ]:
import re

In [ ]:
import string

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import contractions

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from nltk.corpus import wordnet

In [ ]:
from nltk import pos_tag

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
from num2words import num2words

In [ ]:
from spellchecker import SpellChecker

In [ ]:
tweets = pd.read_csv('final.csv').iloc[:,1:]

In [ ]:
tweet = tweets['Tweets']

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
stop_words.remove('not')

In [ ]:
punct_words = list(string.punctuation)

In [ ]:
stop = stop_words + punct_words

In [ ]:
stop = list(set(stop))

In [ ]:
def get_easy_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
spell = SpellChecker()

In [ ]:
digitise = lambda x: re.sub('-|\s+', '', num2words(x.group()))

In [ ]:
def f1(text):
    unurl = re.sub('http[s]?://\S+', '', str(text).strip().lower())
    for i in unurl.split():
        if i[0] in ['#', '@']:
            unurl = unurl.replace(i, ' ')
    unspaced = ' '.join(unurl.split()).strip()
    expanded = contractions.fix(unspaced)
    digit_to_word = re.sub(r'\d+', digitise, expanded)
    clean_words = []
    for word in word_tokenize(digit_to_word):
        if word not in stop:
            pos = pos_tag([word])
            clean_word = lemmatizer.lemmatize(word, get_easy_pos(pos[0][1]))
            clean_words.append(clean_word)
    clean_words = ' '.join(clean_words)
    correct_word = checker.correct(clean_words)
    return correct_word

In [ ]:
tweet0 = tweet[:10]

In [ ]:
st = time.time()
new_tweet0 = tweet0.apply(f1)
print('time_taken : ', time.time() - st)

In [ ]:
len(tweet0), len(new_tweet0)

In [ ]:
st = time.time()
new_tweet = tweet.apply(f1)
print('time_taken : ', time.time() - st)

In [ ]:
print('time for preprocessing in min: ', 3032 / 60)

In [ ]:
tweets['Tweets'] = new_tweet

In [ ]:
tweets.head()

In [ ]:
tweets.iloc[78, 0]

In [ ]:
def phase_1(lines):
    n = len(lines)
    lines_thresh_bool = np.ones((n * n,), dtype=np.bool)
    for i in range(n):
        for j in range(i + 1, n):
            line1 = lines[i]
            n1 = len(line1)
            line2 = lines[j]
            n2 = len(line2)
            n_max = max(n1, n2)
            n_min = min(n1, n2)
            count_true = 0
            for k in range(n_min):
                if line1[k] == line2[k]:
                    count_true += 1
            thresh_true = round(count_true / n_max, 2)
            thresh_false = 1 - thresh_true
            if (thresh_true <= 0.25) or (0.75 <= thresh_false):
                lines_thresh_bool[(n * i) + j] = False
                lines_thresh_bool[i + (n * j)] = False
    return lines_thresh_bool.reshape(n, n)

In [ ]:
def phase_2(lines, lines_thresh_bool):
    tweet_lines_thresh_bool = pd.DataFrame(lines_thresh_bool.T)
    tweet_lines_thresh_bool.insert(0, 'tweet', pd.DataFrame(np.array(lines)))
    l = set()
    for i in range(len(lines)):
        d = tweet_lines_thresh_bool[tweet_lines_thresh_bool[i] == True]['tweet'].to_dict()
        for k in d:
            d[k] = len(d[k])
        d = dict(sorted(d.items(), key=lambda x: x[1], reverse=True))
        l.add(list(d.keys())[0])
    l = list(l)
    refined_tweets = tweet_lines_thresh_bool.loc[l, 'tweet'].to_list()
    return refined_tweets, l

In [ ]:
a_tweets = []
for t in new_tweet:
    a_tweets.append(word_tokenize(str(t).strip()))
len(a_tweets)

In [ ]:
st1 = time.time()
lines_thresh_bool = phase_1(a_tweets)
ed1 = time.time()
st2 = time.time()
refined_tweets, l = phase_2(a_tweets, lines_thresh_bool)
ed2 = time.time()
print('time of phase_1:', ed1 - st1)
print('time of phase_2:', ed2 - st2)
print('time of phase_1 + phase_2:', (ed1 - st1) + (ed2 - st2))
len(tweet), len(refined_tweets)

In [ ]:
plt.figure(figsize=(50,50))
sns.heatmap(lines_thresh_bool)